simple_XRD.jl
by Hezy Amiel
January 2023
Julia 1.8.5

In [15]:
using Plots
using SpecialFunctions

In [16]:
function Gaussian(x, fwhm)
    σ = fwhm/(2√(2log(2)))
    return @. 1/√(2π)/σ * exp(-x^2/2σ^2)
end

Gaussian (generic function with 1 method)

In [17]:
function Lorentzian(x, fwhm)
    γ = fwhm / 2
    return @. (γ/pi) / (x^2 + γ^2)
end

Lorentzian (generic function with 1 method)

In [18]:
function Pseudo_Voigt(x, fwhm, n)
	return n * Lorentzian(x, fwhm) + (1 - n) * Gaussian(x, fwhm)
end

Pseudo_Voigt (generic function with 1 method)

In [19]:
function Voigt(x, fwhm_L, fwhm_G)
    γ = fwhm_L/2
    σ = fwhm_G/(2√(2log(2)))
    z = @. -im * (x + im * γ) / (√2 * σ)
    return @. real(erfcx(z)) / (√(2pi) * σ)
end

Voigt (generic function with 1 method)

In [20]:
function bragg_angels(wavelength, d_spacings)
    sinθ = wavelength ./ (2*d_spacings)
    sinθ_cleand = [item for item in sinθ if abs(sin2θ)<=1]  # removing values outside (-1,1)
    return 2 * (180/π) * asin.(sinθ_cleaned)  # *2 for 2θ  
end

bragg_angels (generic function with 1 method)

In [21]:
function d_list(indices, a)
    return a ./ .√(sum(indices.^2, dims=2))
end

d_list (generic function with 1 method)

In [22]:
N = 1000
θ = LinRange(0, 180, N)

wavelength = 0.15418  # CuKα radiation in nm
#wavelength = 0.18125  # 
U, V, W = 0.2, 0.2, 0.2

(0.2, 0.2, 0.2)

#Simple Cubic
In simple cubic lattince, all Miller indices are allowed

In [40]:
generate_indices = [[h,k,l] for h=0:1 for k=0:1 for l=0:1 if [h,k,l]!=[0,0,0]]
indices_SC = (reduce(hcat, generate_indices))'

7×3 adjoint(::Matrix{Int64}) with eltype Int64:
 0  0  1
 0  1  0
 0  1  1
 1  0  0
 1  0  1
 1  1  0
 1  1  1

In [41]:
d_list(indices_SC, 1)

7×1 Matrix{Float64}:
 1.0
 1.0
 0.7071067811865475
 1.0
 0.7071067811865475
 0.7071067811865475
 0.5773502691896258